In [ ]:
#factrozie labels import numpy as np
import os
import math
import matplotlib.pyplot as plt
from scipy import signal,io,fftpack
import pandas as pd
import mne
from path import Path
import numpy as np
import neurokit2 as nk 
filepath = Path('Results2023/')  
import lime
import lime.lime_tabular

import numpy as np
from tqdm import tqdm_notebook



from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.metrics import *
df_for_classification = pd.read_csv('df_30_classification2023.csv')


In [ ]:

y = df_for_classification.iloc[ :, -1]
X_v=df_for_classification.iloc[:, 2:-1] 
X = X_v.drop(columns = ['MeanNN','MaxNN', 'MinNN'])
#LL 0 , EL 1                               
X                              


In [ ]:


from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.metrics import *


from sklearn.ensemble import GradientBoostingClassifier

# Gradient Boosting Classifier
parameters = {'n_estimators': [1,10,20,30], 'learning_rate' : [0.01,0.05,0.1,0.5],
           'subsample' : [0.1,0.5,1.0], 'max_depth': [1,3,5,10,20,30]}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify =y, random_state=30)    

clf = GridSearchCV(GradientBoostingClassifier(random_state=3000), parameters, cv=5)
clf.fit(X_train, y_train)
print(clf.best_params_)

pred = clf.best_estimator_.predict(X_test)
print("Roc-Auc:", roc_auc_score(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import preprocessing

from sklearn.ensemble import GradientBoostingClassifier



rs = StratifiedShuffleSplit(n_splits = 100, train_size = 0.8, test_size = 0.2, random_state = 0)


roc_auc = []
aucc = []
precision = []
recalls = []
f1s = []
df_lime_weights = []



for i, (train_idx, test_idx) in enumerate(rs.split(X,y)):
    #print(f"Fold : {i}")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    clf.best_estimator_.fit(X_train.values, y_train)
    pred = clf.best_estimator_.predict(X_test.values)
    try:
        roc =roc_auc_score(y_test, pred,average='macro')
        auc = accuracy_score(y_test,pred)
        prc = precision_score(y_true=y_test,y_pred=pred,average='macro')
        rec = recall_score(y_true=y_test,y_pred=pred,average='macro')
        f1 = f1_score(y_true=y_test,y_pred=pred,average='macro')

   
    except ValueError:
        pass
    
    
    roc_auc.append(roc)
    aucc.append(auc)
    precision.append(prc)
    recalls.append(rec)
    f1s.append(f1)
 
    
df_res = pd.DataFrame([aucc, roc_auc, precision, recalls, f1s]).T
df_res.columns = ['Accuracy', 'roc_auc', 'Precision', 'Recall', 'f1']    
 
df_res.to_csv('Results2023/30ClassificationRes/df_30GB.csv', index=False)   


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

#Extra Tree Classifier
parameters = { 'n_estimators': [1,10,100], 
           'criterion' : ['gini', 'entropy'] ,
           'max_depth': [1,5,10,20,], 
           'max_features': ['sqrt','log2'],
           'min_samples_split': [2,5,10],
           'n_jobs': [-1]}


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify =y, random_state=30)    

clf = GridSearchCV( ExtraTreesClassifier(random_state=0), parameters, cv=5)
clf.fit(X_train, y_train)
print(clf.best_params_)



pred = clf.best_estimator_.predict(X_test)
print("Roc-Auc:", roc_auc_score(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import preprocessing



rs = StratifiedShuffleSplit(n_splits = 100, train_size = 0.8, test_size = 0.2, random_state = 0)
 
roc_auc = []
aucc = []
precision = []
recalls = []
f1s = []
df_lime_weights = []


for i, (train_idx, test_idx) in enumerate(rs.split(X,y)):
    #print(f"Fold : {i}")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    clf.best_estimator_.fit(X_train, y_train)
    pred = clf.best_estimator_.predict(X_test)
    try:
        roc =roc_auc_score(y_test, pred,average='macro')
        auc = accuracy_score(y_test,pred)
        prc = precision_score(y_true=y_test,y_pred=pred,average='macro')
        rec = recall_score(y_true=y_test,y_pred=pred,average='macro')
        f1 = f1_score(y_true=y_test,y_pred=pred,average='macro')


   
    except ValueError:
        pass
    
    
    roc_auc.append(roc)
    aucc.append(auc)
    precision.append(prc)
    recalls.append(rec)
    f1s.append(f1)
    
df_res = pd.DataFrame([aucc, roc_auc, precision, recalls, f1s]).T
df_res.columns = ['Accuracy', 'roc_auc', 'Precision', 'Recall', 'f1']    
 
df_res.to_csv('Results2023/30ClassificationRes/df_30ET.csv', index=False)   


In [ ]:

from sklearn.ensemble import RandomForestClassifier
#Random Forest Classifier 
parameters = {'n_estimators': [1,10,100,200],
          'max_depth': [1,5,10,20], 
          'max_features': ['sqrt','log2'],
          'min_samples_split': [2,5,10], 'n_jobs': [-1]}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify =y, random_state=80)    

clf = GridSearchCV(RandomForestClassifier(random_state=0), parameters, cv=5)
clf.fit(X_train, y_train)
print(clf.best_params_)

pred = clf.best_estimator_.predict(X_test)
print("Roc-Auc:", roc_auc_score(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import preprocessing


rs = StratifiedShuffleSplit(n_splits = 100, train_size = 0.8, test_size = 0.2, random_state = 0)


roc_auc = []
aucc = []
precision = []
recalls = []
f1s = []
df_lime_weights = []


for i, (train_idx, test_idx) in enumerate(rs.split(X,y)):
    #print(f"Fold : {i}")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    clf.best_estimator_.fit(X_train, y_train)
    pred = clf.best_estimator_.predict(X_test)
    try:
        roc =roc_auc_score(y_test, pred,average='macro')
        auc = accuracy_score(y_test,pred)
        prc = precision_score(y_true=y_test,y_pred=pred,average='macro')
        rec = recall_score(y_true=y_test,y_pred=pred,average='macro')
        f1 = f1_score(y_true=y_test,y_pred=pred,average='macro')
       



   
    except ValueError:
        pass
    
    
    roc_auc.append(roc)
    aucc.append(auc)
    precision.append(prc)
    recalls.append(rec)
    f1s.append(f1)
    
df_res = pd.DataFrame([aucc, roc_auc, precision, recalls, f1s]).T
df_res.columns = ['Accuracy', 'roc_auc', 'Precision', 'Recall', 'f1']    
 
df_res.to_csv('Results2023/30ClassificationRes/df_30RF.csv', index=False)   


In [ ]:

from sklearn.ensemble import AdaBoostClassifier
# Ada Boost Classifier
parameters = {'algorithm': ['SAMME', 'SAMME.R'],
           'n_estimators': [1,10,100]}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify =y, random_state=30)    

clf = GridSearchCV(AdaBoostClassifier(random_state=0), parameters, cv=5)
clf.fit(X_train, y_train)
print(clf.best_params_)

pred = clf.best_estimator_.predict(X_test)
print("Roc-Auc:", roc_auc_score(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

roc_auc = []
accuracy = []


rs = StratifiedShuffleSplit(n_splits = 100, train_size = 0.50, test_size = 0.50, random_state = 0)



roc_auc = []
aucc = []
precision = []
recalls = []
f1s = []
df_lime_weights = []


for i, (train_idx, test_idx) in enumerate(rs.split(X,y)):
    #print(f"Fold : {i}")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    clf.best_estimator_.fit(X_train, y_train)
    pred = clf.best_estimator_.predict(X_test)
    try:
        roc =roc_auc_score(y_test, pred,average='macro')
        auc = accuracy_score(y_test,pred)
        prc = precision_score(y_true=y_test,y_pred=pred,average='macro')
        rec = recall_score(y_true=y_test,y_pred=pred,average='macro')
        f1 = f1_score(y_true=y_test,y_pred=pred,average='macro')

   
    except ValueError:
        pass
    
    
    roc_auc.append(roc)
    aucc.append(auc)
    precision.append(prc)
    recalls.append(rec)
    f1s.append(f1)
    
df_res = pd.DataFrame([aucc, roc_auc, precision, recalls, f1s]).T
df_res.columns = ['Accuracy', 'roc_auc', 'Precision', 'Recall', 'f1']    
 
df_res.to_csv('Results2023/30ClassificationRes/df_30ADB.csv', index=False)   


In [ ]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
parameters = {'criterion': ['gini', 'entropy'], 
           'max_depth': [1,5,10,20],
           'max_features': [2,5,7],
           'min_samples_split': [2,5,10]}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify =y, random_state=30)    

clf = GridSearchCV(DecisionTreeClassifier(random_state=0), parameters, cv=5)
clf.fit(X_train, y_train)
print(clf.best_params_)

pred = clf.best_estimator_.predict(X_test)
print("Roc-Auc:", roc_auc_score(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import preprocessing
import numpy as np


roc_auc = []
accuracy = []


rs = StratifiedShuffleSplit(n_splits = 100, train_size = 0.8, test_size = 0.2, random_state = 0)

roc_auc = []
aucc = []
precision = []
recalls = []
f1s = []
df_lime_weights = []


for i, (train_idx, test_idx) in enumerate(rs.split(X,y)):
    #print(f"Fold : {i}")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    clf.best_estimator_.fit(X_train, y_train)
    pred = clf.best_estimator_.predict(X_test)
    try:
        roc =roc_auc_score(y_test, pred,average='macro')
        auc = accuracy_score(y_test,pred)
        prc = precision_score(y_true=y_test,y_pred=pred,average='macro')
        rec = recall_score(y_true=y_test,y_pred=pred,average='macro')
        f1 = f1_score(y_true=y_test,y_pred=pred,average='macro')
   
    except ValueError:
        pass
    
    
    roc_auc.append(roc)
    aucc.append(auc)
    precision.append(prc)
    recalls.append(rec)
    f1s.append(f1)
    
df_res = pd.DataFrame([aucc, roc_auc, precision, recalls, f1s]).T
df_res.columns = ['Accuracy', 'roc_auc', 'Precision', 'Recall', 'f1']    

df_res.to_csv('Results2023/30ClassificationRes/df_30DT.csv', index=False)   

print(np.mean(roc_auc)*100.0)
print( np.std(roc_auc)*100.0)

print(np.mean(accuracy)*100.0)
print( np.std(accuracy)*100.0)  
    

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

#KNeighborsClassifier
parameters = {'n_neighbors': [1,5,10],
            'weights': ['uniform','distance'],
            'algorithm': ['auto','ball_tree','kd_tree']}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify =y, random_state=30)    

clf = GridSearchCV(KNeighborsClassifier(), parameters, cv=5)
clf.fit(X_train, y_train)
print(clf.best_params_)

pred = clf.best_estimator_.predict(X_test)
print("Roc-Auc:", roc_auc_score(y_test, pred))
print(classification_report(y_test, pred))


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import preprocessing

from sklearn.ensemble import GradientBoostingClassifier

roc_auc = []
aucc = []


rs = StratifiedShuffleSplit(n_splits = 100, train_size = 0.8, test_size = 0.2, random_state = 0)


roc_auc = []
aucc = []
precision = []
recalls = []
f1s = []
df_lime_weights = []


for i, (train_idx, test_idx) in enumerate(rs.split(X,y)):
    #print(f"Fold : {i}")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    clf.best_estimator_.fit(X_train, y_train)
    pred = clf.best_estimator_.predict(X_test)
    try:
        roc =roc_auc_score(y_test, pred,average='macro')
        auc = accuracy_score(y_test,pred)
        prc = precision_score(y_true=y_test,y_pred=pred,average='macro')
        rec = recall_score(y_true=y_test,y_pred=pred,average='macro')
        f1 = f1_score(y_true=y_test,y_pred=pred,average='macro')
        
    except ValueError:
        pass
    
    
    roc_auc.append(roc)
    aucc.append(auc)
    precision.append(prc)
    recalls.append(rec)
    f1s.append(f1)

    
df_res = pd.DataFrame([aucc, roc_auc, precision, recalls, f1s]).T
df_res.columns = ['Accuracy', 'roc_auc', 'Precision', 'Recall', 'f1']    
 
df_res.to_csv('Results2023/30ClassificationRes/df_30KNN.csv', index=False)   

print(np.mean(roc_auc)*100.0)
print( np.std(roc_auc)*100.0)

print(np.mean(aucc)*100.0)
print( np.std(aucc)*100.0)  

In [ ]:
import xgboost as xgb

parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify =y, random_state=30)    


xgb_classifier = xgb.XGBClassifier()

clf = GridSearchCV(xgb_classifier, parameters, cv=5)
clf.fit(X_train, y_train)
print(clf.best_params_)

pred = clf.best_estimator_.predict(X_test)
print("Roc-Auc:", roc_auc_score(y_test, pred))
print(classification_report(y_test, pred))


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import preprocessing

roc_auc = []
accuracy = []


rs = StratifiedShuffleSplit(n_splits = 100, train_size = 0.8, test_size = 0.2, random_state = 0)


roc_auc = []
aucc = []
precision = []
recalls = []
f1s = []
df_lime_weights = []


for i, (train_idx, test_idx) in enumerate(rs.split(X,y)):
    #print(f"Fold : {i}")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    clf.best_estimator_.fit(X_train, y_train)
    pred = clf.best_estimator_.predict(X_test)
    try:
        roc =roc_auc_score(y_test, pred,average='macro')
        auc = accuracy_score(y_test,pred)
        prc = precision_score(y_true=y_test,y_pred=pred,average='macro')
        rec = recall_score(y_true=y_test,y_pred=pred,average='macro')
        f1 = f1_score(y_true=y_test,y_pred=pred,average='macro')
  



   
    except ValueError:
        pass
    
    
    roc_auc.append(roc)
    aucc.append(auc)
    precision.append(prc)
    recalls.append(rec)
    f1s.append(f1)
    
df_res = pd.DataFrame([aucc, roc_auc, precision, recalls, f1s]).T
df_res.columns = ['Accuracy', 'roc_auc', 'Precision', 'Recall', 'f1']    
 

df_res.to_csv('Results2023/30ClassificationRes/df_30XGB.csv', index=False)   

    

print(np.mean(roc_auc)*100.0)
print( np.std(roc_auc)*100.0)

print(np.mean(accuracy)*100.0)
print( np.std(accuracy)*100.0)    

In [ ]:
#MLPClassifier
from sklearn.neural_network import MLPClassifier




parameters = {'hidden_layer_sizes': [(100,200,), (10)],
                'activation': ['tanh', 'relu', 'logistic'],
                'solver': ['sgd', 'adam', 'lbfgs'],
                'alpha': [0, 0.0001, 0.05],
                'learning_rate': ['constant','adaptive']
}


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify =y, random_state=30)    


mlp = MLPClassifier(max_iter= 100, random_state=0) 

clf = GridSearchCV(mlp, parameters, cv=5)
clf.fit(X_train, y_train)
print(clf.best_params_)

pred = clf.best_estimator_.predict(X_test)
print("Roc-Auc:", roc_auc_score(y_test, pred))
print(classification_report(y_test, pred))


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import preprocessing

roc_auc = []


rs = StratifiedShuffleSplit(n_splits = 100, train_size = 0.8, test_size = 0.2, random_state = 0)


roc_auc = []
aucc = []
precision = []
recalls = []
f1s = []
df_lime_weights = []


for i, (train_idx, test_idx) in enumerate(rs.split(X,y)):
    #print(f"Fold : {i}")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    clf.best_estimator_.fit(X_train, y_train)
    pred = clf.best_estimator_.predict(X_test)
    try:
        roc =roc_auc_score(y_test, pred,average='macro')
        auc = accuracy_score(y_test,pred)
        prc = precision_score(y_true=y_test,y_pred=pred,average='macro')
        rec = recall_score(y_true=y_test,y_pred=pred,average='macro')
        f1 = f1_score(y_true=y_test,y_pred=pred,average='macro')
        


   
    except ValueError:
        pass
    
    
    roc_auc.append(roc)
    aucc.append(auc)
    precision.append(prc)
    recalls.append(rec)
    f1s.append(f1)
    
df_res = pd.DataFrame([aucc, roc_auc, precision, recalls, f1s]).T
df_res.columns = ['Accuracy', 'roc_auc', 'Precision', 'Recall', 'f1']    
 


df_res.to_csv('Results2023/30ClassificationRes/df_30MLP.csv', index=False)   

    

print(np.mean(roc_auc)*100.0)
print( np.std(roc_auc)*100.0)

print(np.mean(accuracy)*100.0)
print( np.std(accuracy)*100.0)    